In [ ]:
import json
from datetime import datetime
import calendar
from collections import defaultdict

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d").date()

def get_month_range(month_str):
    year, month = map(int, month_str.split('-'))
    start_date = datetime(year, month, 1).date()
    _, last_day = calendar.monthrange(year, month)
    end_date = datetime(year, month, last_day).date()
    return start_date, end_date

def calculate_active_days(item_start, item_stop, month_start, month_end):
    active_start = max(item_start, month_start)
    active_end = min(item_stop, month_end)
    if active_start > active_end:
        return 0, None, None
    return (active_end - active_start).days + 1, active_start, active_end

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    month_start, month_end = get_month_range(target_month)
    
    grouped_items = defaultdict(lambda: {"qty": 0, "amount": 0.0})
    
    for item in item_list:
        item_start = parse_date(item["start_date"])
        item_stop = parse_date(item["stop_date"])
        
        active_days, active_start, active_end = calculate_active_days(item_start, item_stop, month_start, month_end)
        if active_days == 0:
            continue

        item_code = item.get("item_code")
        qty = int(item.get("qty", 0))
        rate = float(item.get("rate", 0))
        
        total_days_in_month = (month_end - month_start).days + 1
        daily_rate = rate / total_days_in_month
        prorated_amount = round(qty * daily_rate * active_days, 2)

        billing_period_str = f"{active_start} to {active_end}"
        grouping_key = (item_code, rate, billing_period_str)
        
        grouped_items[grouping_key]["qty"] += qty
        grouped_items[grouping_key]["amount"] += prorated_amount

    line_items = []
    total_revenue = 0.0
    
    for (item_code, rate, billing_period), data in grouped_items.items():
        amount = round(data["amount"], 2)
        total_revenue += amount
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": data["qty"],
            "amount": amount,
            "billing_period": billing_period
        })
    
    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

if __name__ == "__main__":
    with open("item_list.json", "r") as file:
        items = json.load(file)
    
    result = generate_monthly_bill(items, "2024-11")
    
    from pprint import pprint
    pprint(result)
